In [1]:
from related_generator.post import Post, get_all_posts
import nltk
import umap
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
import graphviz
import json

from bs4 import BeautifulSoup


/home/wint3rmute/code/web/baczek.me/.venv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/wint3rmute/code/web/baczek.me/.venv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/wint3rmute/code/web/baczek.me/.venv/lib/python3.11

In [2]:
all_posts = get_all_posts()

/home/wint3rmute/code/web/baczek.me/related_generator/post.py:46: UserWarning: No tags defined for content/2023-05-02-11-01-19.md
  warnings.warn(f"No tags defined for {file_path}")
/home/wint3rmute/code/web/baczek.me/related_generator/post.py:46: UserWarning: No tags defined for content/should_listen.md
  warnings.warn(f"No tags defined for {file_path}")


In [3]:
def tokenizer(text: str) -> list[str]:
    stemmer = SnowballStemmer("english")
    words = word_tokenize(text)

    return [stemmer.stem(word) for word in words]

In [4]:
# for post in all_posts:
#     html_path = (Path("./public") / post.path.stem) / "index.html"
#     html_path = str(html_path).replace("_", "-")
#     html_content = open(html_path).read()
    
#     html_tree = BeautifulSoup(html_content, features="html.parser")
#     content = html_tree.article.text.replace("\n", " ")
#     post._old_content = post.content
#     post.content = content


In [5]:
all_posts[1].content

'\ntitle: Decentralisation\ntags: programming\n\nMaking the internet interesting again\n{{image (src="/computer_poster.jpg" small=true title="")}}\n\nCheck out my bookmarks!\nFLOSS software\nMy recommendations on open source software.\nDecentralised/self-hosted online services\nI\'m running self-hosted online services. They are public, you can use them too :)\n\n\nInvidious, an alternate YouTube frontend\n\n\nSearX, a meta-search engine\n\n\nLibrex, another meta-search engine\n\n\nLibreddit, alternative Reddit frontend\n\n\nI also recommend the following services for private use:\n\n\nNextCloud\n\n\nMatrix \n\n\n\nnew idea, work in progress - registry-based search engine manifesto\n\nNotes on self-hosting\n\nMonitor the metrics of every service you\'ve hosting (Prometheus)\nVisualise your metrics (Grafana)\nPing everything from time to time (Blackbox exporter)\nSetup notifications when a service goes down\nAutomate everything from the start, you won\'t regret it\nAs your infrastructure

In [9]:

# Vectorizer to convert a collection of raw documents to a matrix of TF-IDF features
vectorizer = TfidfVectorizer(tokenizer=tokenizer)
# vectorizer = TfidfVectorizer()

# Learn vocabulary and idf, return term-document matrix.
tfidf = vectorizer.fit_transform([post.content for post in all_posts])

# Array mapping from feature integer indices to feature name
words = vectorizer.get_feature_names_out()

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode([post.content for post in all_posts])


# umap_result = umap.UMAP().fit_transform(tfidf)
umap_result = umap.UMAP().fit_transform(embeddings)

# for post, umap_result in zip(all_posts, umap_result):
#     post.x, post.y = umap_result
    
#     plt.scatter(post.x, post.y)



/home/wint3rmute/code/web/baczek.me/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [11]:
import matplotlib.pyplot as plt

for post, umap_result in zip(all_posts, umap_result):
    post.x, post.y = umap_result
    
    plt.scatter(post.x, post.y, text=post.title)


TypeError: cannot unpack non-iterable numpy.float32 object